# Uncertainty-Weighted Ensemble with Conservative Predictions

**Hypothesis**: The high CV-LB intercept (0.0533 > target 0.0347) suggests the model is overconfident on OOD samples. By using GP uncertainty to identify high-uncertainty predictions and making them more conservative (closer to training mean), we might reduce the intercept.

**Key Insight**:
- CV-LB relationship: LB = 4.23 × CV + 0.0533 (R² = 0.98)
- Intercept (0.0533) > Target (0.0347) - even with CV=0, predicted LB would be 0.0533
- The problem is OOD generalization - we need to be more conservative on uncertain predictions

**Implementation**:
1. Get GP predictions and uncertainty (std)
2. Get ensemble predictions (GP + MLP + LGBM)
3. For high-uncertainty samples, blend toward training mean
4. High uncertainty → more weight to training mean (conservative)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
import tqdm
import warnings
warnings.filterwarnings('ignore')

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

# Simple Featurizer (for GP) - 18 features
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')
print(f'Simple feature dimension: {SimpleFeaturizer().feats_dim}')

Full feature dimension: 145
Simple feature dimension: 18


In [5]:
# MLP Model
class MLPModel(nn.Module):
    def __init__(self, input_dim=145, hidden_dim=64, output_dim=3, dropout=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

print('MLP model defined')

MLP model defined


In [6]:
# Uncertainty-Weighted Ensemble Model
class UncertaintyWeightedEnsemble:
    """
    GP + MLP + LGBM ensemble with uncertainty-weighted conservative predictions.
    
    For high-uncertainty (OOD) samples, blend predictions toward training mean
    to reduce overconfident extrapolation errors.
    """
    def __init__(self, data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.30, 
                 uncertainty_blend=0.3):
        self.data = data
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        self.uncertainty_blend = uncertainty_blend  # How much to blend toward mean for high uncertainty
        self.mixed = (data == 'full')
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        self.gp_models = []
        self.mlp_model = None
        self.lgbm_models = []
        self.train_mean = None  # Store training mean for conservative predictions
        
    def train_model(self, X, Y):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        # Store training mean for conservative predictions
        self.train_mean = Y_np.mean(axis=0)
        
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        
        # Train GP for each target (with return_std capability)
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(
                kernel=kernel, alpha=0.01, n_restarts_optimizer=3, random_state=42
            )
            gp.fit(X_simple_scaled, Y_np[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        self.mlp_model = MLPModel(
            input_dim=self.full_featurizer.feats_dim,
            hidden_dim=64,
            output_dim=3,
            dropout=0.2
        ).to(device)
        
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_np, dtype=torch.double).to(device)
        
        optimizer = torch.optim.Adam(self.mlp_model.parameters(), lr=0.001, weight_decay=1e-4)
        criterion = nn.MSELoss()
        
        self.mlp_model.train()
        for epoch in range(200):
            optimizer.zero_grad()
            pred = self.mlp_model(X_tensor)
            loss = criterion(pred, Y_tensor)
            loss.backward()
            optimizer.step()
        
        # Train LGBM for each target
        for i in range(3):
            lgbm = lgb.LGBMRegressor(
                n_estimators=100, learning_rate=0.05, max_depth=5,
                num_leaves=31, min_child_samples=5, random_state=42, verbose=-1
            )
            lgbm.fit(X_full_scaled, Y_np[:, i])
            self.lgbm_models.append(lgbm)
        
        return self
    
    def predict(self, X):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # GP predictions with uncertainty
        gp_preds = []
        gp_stds = []
        for i in range(3):
            pred, std = self.gp_models[i].predict(X_simple_scaled, return_std=True)
            gp_preds.append(pred)
            gp_stds.append(std)
        gp_preds = np.column_stack(gp_preds)
        gp_stds = np.column_stack(gp_stds)
        
        # MLP predictions
        self.mlp_model.eval()
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        with torch.no_grad():
            mlp_preds = self.mlp_model(X_tensor).cpu().numpy()
        
        # LGBM predictions
        lgbm_preds = np.column_stack([self.lgbm_models[i].predict(X_full_scaled) for i in range(3)])
        
        # Standard ensemble prediction
        ensemble_preds = self.gp_weight * gp_preds + self.mlp_weight * mlp_preds + self.lgbm_weight * lgbm_preds
        
        # Compute uncertainty weight (normalized std across all targets)
        avg_std = gp_stds.mean(axis=1, keepdims=True)
        max_std = avg_std.max() if avg_std.max() > 0 else 1.0
        uncertainty_weight = np.clip(avg_std / max_std, 0, 1) * self.uncertainty_blend
        
        # Blend toward training mean for high-uncertainty samples
        predictions = (1 - uncertainty_weight) * ensemble_preds + uncertainty_weight * self.train_mean
        
        # TTA for mixtures
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
            
            X_full_scaled_flip = self.scaler_full.transform(X_full_flip)
            X_simple_scaled_flip = self.scaler_simple.transform(X_simple_flip)
            
            gp_preds_flip = []
            gp_stds_flip = []
            for i in range(3):
                pred, std = self.gp_models[i].predict(X_simple_scaled_flip, return_std=True)
                gp_preds_flip.append(pred)
                gp_stds_flip.append(std)
            gp_preds_flip = np.column_stack(gp_preds_flip)
            gp_stds_flip = np.column_stack(gp_stds_flip)
            
            X_tensor_flip = torch.tensor(X_full_scaled_flip, dtype=torch.double).to(device)
            with torch.no_grad():
                mlp_preds_flip = self.mlp_model(X_tensor_flip).cpu().numpy()
            lgbm_preds_flip = np.column_stack([self.lgbm_models[i].predict(X_full_scaled_flip) for i in range(3)])
            
            ensemble_preds_flip = self.gp_weight * gp_preds_flip + self.mlp_weight * mlp_preds_flip + self.lgbm_weight * lgbm_preds_flip
            
            avg_std_flip = gp_stds_flip.mean(axis=1, keepdims=True)
            uncertainty_weight_flip = np.clip(avg_std_flip / max_std, 0, 1) * self.uncertainty_blend
            predictions_flip = (1 - uncertainty_weight_flip) * ensemble_preds_flip + uncertainty_weight_flip * self.train_mean
            
            predictions = (predictions + predictions_flip) / 2
        
        predictions = np.clip(predictions, 0, 1)
        return torch.tensor(predictions)

print('Uncertainty-Weighted Ensemble defined')

Uncertainty-Weighted Ensemble defined


In [7]:
# Quick test
X_single, Y_single = load_data("single_solvent")
print(f'Single solvent data: X={X_single.shape}, Y={Y_single.shape}')

# Test on a small subset
X_train = X_single.iloc[:80]
Y_train = Y_single.iloc[:80]
X_test = X_single.iloc[80:100]
Y_test = Y_single.iloc[80:100]

model = UncertaintyWeightedEnsemble(data='single', uncertainty_blend=0.3)
model.train_model(X_train, Y_train)
preds = model.predict(X_test)
print(f'Test predictions shape: {preds.shape}')
print(f'Test predictions range: [{preds.min():.4f}, {preds.max():.4f}]')
print(f'Training mean: {model.train_mean}')

Single solvent data: X=(656, 3), Y=(656, 3)


Test predictions shape: torch.Size([20, 3])
Test predictions range: [0.1733, 0.5232]
Training mean: [0.18109954 0.19224699 0.48085667]


In [11]:
# Run CV on single solvent data - Try lower blend factor
print('\n=== Single Solvent CV (Uncertainty-Weighted, blend=0.05) ===')
X_single, Y_single = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = UncertaintyWeightedEnsemble(data='single', uncertainty_blend=0.05)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_single.append(predictions.numpy())
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_predictions_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((preds_single - actuals_single) ** 2)
print(f'Single Solvent MSE: {mse_single:.6f} (n={len(preds_single)})')


=== Single Solvent CV (Uncertainty-Weighted, blend=0.05) ===


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:20<07:53, 20.60s/it]

  8%|▊         | 2/24 [00:40<07:23, 20.15s/it]

 12%|█▎        | 3/24 [00:58<06:44, 19.24s/it]

...

 92%|█████████▏| 22/24 [07:18<00:39, 19.77s/it]

 96%|█████████▌| 23/24 [07:40<00:20, 20.32s/it]

100%|██████████| 24/24 [07:59<00:00, 19.87s/it]

100%|██████████| 24/24 [07:59<00:00, 19.96s/it]

Single Solvent MSE: 0.010652 (n=656)


In [12]:
# Run CV on full data - Try lower blend factor
print('\n=== Full Data CV (Uncertainty-Weighted, blend=0.05) ===')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = UncertaintyWeightedEnsemble(data='full', uncertainty_blend=0.05)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_full.append(predictions.numpy())
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((preds_full - actuals_full) ** 2)
print(f'Full Data MSE: {mse_full:.6f} (n={len(preds_full)})')


=== Full Data CV (Uncertainty-Weighted, blend=0.05) ===


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [01:40<20:03, 100.26s/it]

 15%|█▌        | 2/13 [03:15<17:49, 97.24s/it] 

 23%|██▎       | 3/13 [04:47<15:51, 95.12s/it]

...

 85%|████████▍ | 11/13 [16:39<03:07, 93.81s/it]

 92%|█████████▏| 12/13 [18:22<01:36, 96.77s/it]

100%|██████████| 13/13 [19:54<00:00, 95.37s/it]

100%|██████████| 13/13 [19:54<00:00, 91.92s/it]

Full Data MSE: 0.010029 (n=1227)


In [13]:
# Calculate overall MSE
n_single = len(preds_single)
n_full = len(preds_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE SUMMARY (Uncertainty-Weighted, blend=0.3) ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')

# Predict LB using CV-LB relationship
predicted_lb = 4.23 * overall_mse + 0.0533
print(f'\nPredicted LB (using LB = 4.23*CV + 0.0533): {predicted_lb:.4f}')
print(f'Best LB achieved: 0.0877')
print(f'Target: 0.0347')


=== CV SCORE SUMMARY (Uncertainty-Weighted, blend=0.3) ===
Single Solvent MSE: 0.010652 (n=656)
Full Data MSE: 0.010029 (n=1227)
Overall MSE: 0.010246

Best CV (exp_032): 0.008194

✗ WORSE: 25.04% worse than best CV

Predicted LB (using LB = 4.23*CV + 0.0533): 0.0966
Best LB achieved: 0.0877
Target: 0.0347


In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = UncertaintyWeightedEnsemble(data='single', uncertainty_blend=0.3)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = UncertaintyWeightedEnsemble(data='full', uncertainty_blend=0.3)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Final verification
print(f'\n=== FINAL CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')
print(f'Predicted LB: {predicted_lb:.4f}')
print(f'Target: 0.0347')

In [14]:
# Try Nearest-Neighbor Blending approach
# For OOD samples, predictions from the most similar training solvents might be more reliable

from sklearn.neighbors import NearestNeighbors

class NearestNeighborBlendingEnsemble:
    """
    GP + MLP + LGBM ensemble with nearest-neighbor blending.
    
    For samples far from training data, blend predictions with nearest neighbor predictions.
    """
    def __init__(self, data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.30, 
                 nn_blend_temp=1.0, k_neighbors=3):
        self.data = data
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        self.nn_blend_temp = nn_blend_temp  # Temperature for distance-based blending
        self.k_neighbors = k_neighbors
        self.mixed = (data == 'full')
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        self.gp_models = []
        self.mlp_model = None
        self.lgbm_models = []
        self.nn_model = None
        self.train_Y = None
        self.train_X_scaled = None
        
    def train_model(self, X, Y):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        self.train_Y = Y_np
        
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        
        self.train_X_scaled = X_full_scaled
        
        # Train nearest neighbor model
        self.nn_model = NearestNeighbors(n_neighbors=self.k_neighbors, metric='euclidean')
        self.nn_model.fit(X_full_scaled)
        
        # Train GP for each target
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(
                kernel=kernel, alpha=0.01, n_restarts_optimizer=3, random_state=42
            )
            gp.fit(X_simple_scaled, Y_np[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        self.mlp_model = MLPModel(
            input_dim=self.full_featurizer.feats_dim,
            hidden_dim=64,
            output_dim=3,
            dropout=0.2
        ).to(device)
        
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_np, dtype=torch.double).to(device)
        
        optimizer = torch.optim.Adam(self.mlp_model.parameters(), lr=0.001, weight_decay=1e-4)
        criterion = nn.MSELoss()
        
        self.mlp_model.train()
        for epoch in range(200):
            optimizer.zero_grad()
            pred = self.mlp_model(X_tensor)
            loss = criterion(pred, Y_tensor)
            loss.backward()
            optimizer.step()
        
        # Train LGBM for each target
        for i in range(3):
            lgbm = lgb.LGBMRegressor(
                n_estimators=100, learning_rate=0.05, max_depth=5,
                num_leaves=31, min_child_samples=5, random_state=42, verbose=-1
            )
            lgbm.fit(X_full_scaled, Y_np[:, i])
            self.lgbm_models.append(lgbm)
        
        return self
    
    def predict(self, X):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # GP predictions
        gp_preds = np.column_stack([self.gp_models[i].predict(X_simple_scaled) for i in range(3)])
        
        # MLP predictions
        self.mlp_model.eval()
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        with torch.no_grad():
            mlp_preds = self.mlp_model(X_tensor).cpu().numpy()
        
        # LGBM predictions
        lgbm_preds = np.column_stack([self.lgbm_models[i].predict(X_full_scaled) for i in range(3)])
        
        # Standard ensemble prediction
        ensemble_preds = self.gp_weight * gp_preds + self.mlp_weight * mlp_preds + self.lgbm_weight * lgbm_preds
        
        # Get nearest neighbor predictions
        distances, indices = self.nn_model.kneighbors(X_full_scaled)
        nn_preds = self.train_Y[indices].mean(axis=1)  # Average of k nearest neighbors
        
        # Blend based on distance (closer = more weight to model, farther = more weight to NN)
        avg_distance = distances.mean(axis=1, keepdims=True)
        # Normalize by training set distances
        train_distances, _ = self.nn_model.kneighbors(self.train_X_scaled)
        train_avg_dist = train_distances.mean()
        
        # Blend weight: 0 = use model, 1 = use NN
        blend_weight = 1 - np.exp(-avg_distance / (self.nn_blend_temp * train_avg_dist))
        
        predictions = (1 - blend_weight) * ensemble_preds + blend_weight * nn_preds
        
        # TTA for mixtures
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
            
            X_full_scaled_flip = self.scaler_full.transform(X_full_flip)
            X_simple_scaled_flip = self.scaler_simple.transform(X_simple_flip)
            
            gp_preds_flip = np.column_stack([self.gp_models[i].predict(X_simple_scaled_flip) for i in range(3)])
            
            X_tensor_flip = torch.tensor(X_full_scaled_flip, dtype=torch.double).to(device)
            with torch.no_grad():
                mlp_preds_flip = self.mlp_model(X_tensor_flip).cpu().numpy()
            lgbm_preds_flip = np.column_stack([self.lgbm_models[i].predict(X_full_scaled_flip) for i in range(3)])
            
            ensemble_preds_flip = self.gp_weight * gp_preds_flip + self.mlp_weight * mlp_preds_flip + self.lgbm_weight * lgbm_preds_flip
            
            distances_flip, indices_flip = self.nn_model.kneighbors(X_full_scaled_flip)
            nn_preds_flip = self.train_Y[indices_flip].mean(axis=1)
            avg_distance_flip = distances_flip.mean(axis=1, keepdims=True)
            blend_weight_flip = 1 - np.exp(-avg_distance_flip / (self.nn_blend_temp * train_avg_dist))
            
            predictions_flip = (1 - blend_weight_flip) * ensemble_preds_flip + blend_weight_flip * nn_preds_flip
            
            predictions = (predictions + predictions_flip) / 2
        
        predictions = np.clip(predictions, 0, 1)
        return torch.tensor(predictions)

print('Nearest-Neighbor Blending Ensemble defined')

Nearest-Neighbor Blending Ensemble defined


In [15]:
# Quick test of NN blending
print('Testing NN Blending Ensemble...')
X_train = X_single.iloc[:80]
Y_train = Y_single.iloc[:80]
X_test = X_single.iloc[80:100]

model = NearestNeighborBlendingEnsemble(data='single', nn_blend_temp=1.0, k_neighbors=3)
model.train_model(X_train, Y_train)
preds = model.predict(X_test)
print(f'Test predictions shape: {preds.shape}')

Testing NN Blending Ensemble...


Test predictions shape: torch.Size([20, 3])


In [16]:
# Run CV on single solvent data with NN blending
print('\n=== Single Solvent CV (NN Blending, temp=1.0) ===')
X_single, Y_single = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single_nn = []
all_actuals_single_nn = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = NearestNeighborBlendingEnsemble(data='single', nn_blend_temp=1.0, k_neighbors=3)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_single_nn.append(predictions.numpy())
    all_actuals_single_nn.append(test_Y.values)

preds_single_nn = np.vstack(all_predictions_single_nn)
actuals_single_nn = np.vstack(all_actuals_single_nn)
mse_single_nn = np.mean((preds_single_nn - actuals_single_nn) ** 2)
print(f'Single Solvent MSE: {mse_single_nn:.6f} (n={len(preds_single_nn)})')


=== Single Solvent CV (NN Blending, temp=1.0) ===


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:24<09:34, 24.99s/it]

  8%|▊         | 2/24 [00:45<08:07, 22.18s/it]

 12%|█▎        | 3/24 [01:03<07:08, 20.42s/it]

...

 92%|█████████▏| 22/24 [07:23<00:39, 19.70s/it]

 96%|█████████▌| 23/24 [07:45<00:20, 20.50s/it]

100%|██████████| 24/24 [08:04<00:00, 20.17s/it]

100%|██████████| 24/24 [08:04<00:00, 20.21s/it]

Single Solvent MSE: 0.033500 (n=656)


In [17]:
# Run CV on full data with NN blending
print('\n=== Full Data CV (NN Blending, temp=1.0) ===')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full_nn = []
all_actuals_full_nn = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = NearestNeighborBlendingEnsemble(data='full', nn_blend_temp=1.0, k_neighbors=3)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_full_nn.append(predictions.numpy())
    all_actuals_full_nn.append(test_Y.values)

preds_full_nn = np.vstack(all_predictions_full_nn)
actuals_full_nn = np.vstack(all_actuals_full_nn)
mse_full_nn = np.mean((preds_full_nn - actuals_full_nn) ** 2)
print(f'Full Data MSE: {mse_full_nn:.6f} (n={len(preds_full_nn)})')


=== Full Data CV (NN Blending, temp=1.0) ===


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [01:40<20:04, 100.40s/it]

 15%|█▌        | 2/13 [03:15<17:53, 97.57s/it] 

 23%|██▎       | 3/13 [04:48<15:54, 95.41s/it]

...

 85%|████████▍ | 11/13 [16:41<03:07, 93.89s/it]

 92%|█████████▏| 12/13 [18:24<01:36, 96.74s/it]

100%|██████████| 13/13 [19:56<00:00, 95.38s/it]

100%|██████████| 13/13 [19:56<00:00, 92.05s/it]

Full Data MSE: 0.024093 (n=1227)


In [18]:
# Calculate overall MSE for NN blending
n_single_nn = len(preds_single_nn)
n_full_nn = len(preds_full_nn)
overall_mse_nn = (mse_single_nn * n_single_nn + mse_full_nn * n_full_nn) / (n_single_nn + n_full_nn)

print(f'\n=== CV SCORE SUMMARY (NN Blending, temp=1.0) ===')
print(f'Single Solvent MSE: {mse_single_nn:.6f} (n={n_single_nn})')
print(f'Full Data MSE: {mse_full_nn:.6f} (n={n_full_nn})')
print(f'Overall MSE: {overall_mse_nn:.6f}')
print(f'\nBest CV (exp_032): 0.008194')

if overall_mse_nn < 0.008194:
    improvement = (0.008194 - overall_mse_nn) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse_nn - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')

# Predict LB using CV-LB relationship
predicted_lb_nn = 4.23 * overall_mse_nn + 0.0533
print(f'\nPredicted LB (using LB = 4.23*CV + 0.0533): {predicted_lb_nn:.4f}')


=== CV SCORE SUMMARY (NN Blending, temp=1.0) ===
Single Solvent MSE: 0.033500 (n=656)
Full Data MSE: 0.024093 (n=1227)
Overall MSE: 0.027371

Best CV (exp_032): 0.008194

✗ WORSE: 234.03% worse than best CV

Predicted LB (using LB = 4.23*CV + 0.0533): 0.1691


In [19]:
# Try pure GP model to see if it has different CV-LB relationship
# GP might have better OOD generalization due to its uncertainty-aware nature

class PureGPEnsemble:
    """
    Pure GP model with multi-output prediction.
    """
    def __init__(self, data='single'):
        self.data = data
        self.mixed = (data == 'full')
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        self.scaler = StandardScaler()
        self.gp_models = []
        
    def train_model(self, X, Y):
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        X_scaled = self.scaler.fit_transform(X_simple)
        
        # Train GP for each target
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(
                kernel=kernel, alpha=0.01, n_restarts_optimizer=3, random_state=42
            )
            gp.fit(X_scaled, Y_np[:, i])
            self.gp_models.append(gp)
        
        return self
    
    def predict(self, X):
        X_simple = self.simple_featurizer.featurize(X)
        X_scaled = self.scaler.transform(X_simple)
        
        gp_preds = np.column_stack([self.gp_models[i].predict(X_scaled) for i in range(3)])
        
        # TTA for mixtures
        if self.mixed:
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
            X_scaled_flip = self.scaler.transform(X_simple_flip)
            gp_preds_flip = np.column_stack([self.gp_models[i].predict(X_scaled_flip) for i in range(3)])
            gp_preds = (gp_preds + gp_preds_flip) / 2
        
        predictions = np.clip(gp_preds, 0, 1)
        return torch.tensor(predictions)

print('Pure GP Ensemble defined')

Pure GP Ensemble defined


In [20]:
# Run CV on single solvent data with pure GP
print('\n=== Single Solvent CV (Pure GP) ===')
X_single, Y_single = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single_gp = []
all_actuals_single_gp = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = PureGPEnsemble(data='single')
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_single_gp.append(predictions.numpy())
    all_actuals_single_gp.append(test_Y.values)

preds_single_gp = np.vstack(all_predictions_single_gp)
actuals_single_gp = np.vstack(all_actuals_single_gp)
mse_single_gp = np.mean((preds_single_gp - actuals_single_gp) ** 2)
print(f'Single Solvent MSE: {mse_single_gp:.6f} (n={len(preds_single_gp)})')


=== Single Solvent CV (Pure GP) ===


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:20<07:54, 20.63s/it]

  8%|▊         | 2/24 [00:40<07:20, 20.03s/it]

 12%|█▎        | 3/24 [00:58<06:39, 19.02s/it]

...

 92%|█████████▏| 22/24 [07:14<00:41, 20.66s/it]

 96%|█████████▌| 23/24 [07:36<00:21, 21.06s/it]

100%|██████████| 24/24 [07:56<00:00, 20.51s/it]

100%|██████████| 24/24 [07:56<00:00, 19.84s/it]

Single Solvent MSE: 0.011722 (n=656)


In [21]:
# Run CV on full data with pure GP
print('\n=== Full Data CV (Pure GP) ===')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full_gp = []
all_actuals_full_gp = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = PureGPEnsemble(data='full')
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_full_gp.append(predictions.numpy())
    all_actuals_full_gp.append(test_Y.values)

preds_full_gp = np.vstack(all_predictions_full_gp)
actuals_full_gp = np.vstack(all_actuals_full_gp)
mse_full_gp = np.mean((preds_full_gp - actuals_full_gp) ** 2)
print(f'Full Data MSE: {mse_full_gp:.6f} (n={len(preds_full_gp)})')


=== Full Data CV (Pure GP) ===


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [01:39<19:56, 99.72s/it]

 15%|█▌        | 2/13 [03:14<17:45, 96.83s/it]

 23%|██▎       | 3/13 [04:46<15:46, 94.67s/it]

...

 85%|████████▍ | 11/13 [16:33<03:06, 93.18s/it]

 92%|█████████▏| 12/13 [18:15<01:36, 96.02s/it]

100%|██████████| 13/13 [19:47<00:00, 94.67s/it]

100%|██████████| 13/13 [19:47<00:00, 91.35s/it]

Full Data MSE: 0.010991 (n=1227)


In [22]:
# Calculate overall MSE for pure GP
n_single_gp = len(preds_single_gp)
n_full_gp = len(preds_full_gp)
overall_mse_gp = (mse_single_gp * n_single_gp + mse_full_gp * n_full_gp) / (n_single_gp + n_full_gp)

print(f'\n=== CV SCORE SUMMARY (Pure GP) ===')
print(f'Single Solvent MSE: {mse_single_gp:.6f} (n={n_single_gp})')
print(f'Full Data MSE: {mse_full_gp:.6f} (n={n_full_gp})')
print(f'Overall MSE: {overall_mse_gp:.6f}')
print(f'\nBest CV (exp_032): 0.008194')

if overall_mse_gp < 0.008194:
    improvement = (0.008194 - overall_mse_gp) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse_gp - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')

# Predict LB using CV-LB relationship
predicted_lb_gp = 4.23 * overall_mse_gp + 0.0533
print(f'\nPredicted LB (using LB = 4.23*CV + 0.0533): {predicted_lb_gp:.4f}')


=== CV SCORE SUMMARY (Pure GP) ===
Single Solvent MSE: 0.011722 (n=656)
Full Data MSE: 0.010991 (n=1227)
Overall MSE: 0.011246

Best CV (exp_032): 0.008194

✗ WORSE: 37.24% worse than best CV

Predicted LB (using LB = 4.23*CV + 0.0533): 0.1009


In [23]:
# Summary of all approaches tried in this experiment
print('\\n=== EXPERIMENT 065 SUMMARY ===')
print('\\nApproach 1: Uncertainty-Weighted (blend=0.3)')
print(f'  Overall MSE: 0.012241 (49.39% worse than baseline)')

print('\\nApproach 2: Uncertainty-Weighted (blend=0.05)')
print(f'  Overall MSE: 0.010246 (25.04% worse than baseline)')

print('\\nApproach 3: NN Blending (temp=1.0)')
print(f'  Overall MSE: {overall_mse_nn:.6f} (234.03% worse than baseline)')

print('\\nApproach 4: Pure GP')
print(f'  Overall MSE: {overall_mse_gp:.6f} (37.24% worse than baseline)')

print('\\n=== CONCLUSION ===')
print('All approaches tried in this experiment performed WORSE than baseline.')
print('The baseline GP+MLP+LGBM ensemble (CV=0.008194) remains the best.')
print('\\nKey insights:')
print('1. Conservative predictions (blending toward mean) hurt CV performance')
print('2. NN blending hurts performance significantly')
print('3. Pure GP is worse than the ensemble')
print('4. The ensemble diversity is valuable - removing components hurts performance')

# Best score from this experiment
best_cv_this_exp = min(0.012241, 0.010246, overall_mse_nn, overall_mse_gp)
print(f'\\nBest CV from this experiment: {best_cv_this_exp:.6f}')

\n=== EXPERIMENT 065 SUMMARY ===
\nApproach 1: Uncertainty-Weighted (blend=0.3)
  Overall MSE: 0.012241 (49.39% worse than baseline)
\nApproach 2: Uncertainty-Weighted (blend=0.05)
  Overall MSE: 0.010246 (25.04% worse than baseline)
\nApproach 3: NN Blending (temp=1.0)
  Overall MSE: 0.027371 (234.03% worse than baseline)
\nApproach 4: Pure GP
  Overall MSE: 0.011246 (37.24% worse than baseline)
\n=== CONCLUSION ===
All approaches tried in this experiment performed WORSE than baseline.
The baseline GP+MLP+LGBM ensemble (CV=0.008194) remains the best.
\nKey insights:
1. Conservative predictions (blending toward mean) hurt CV performance
2. NN blending hurts performance significantly
3. Pure GP is worse than the ensemble
4. The ensemble diversity is valuable - removing components hurts performance
\nBest CV from this experiment: 0.010246
